
##### Extra : Random Forest 

Dataset disponibles :
- X_train : avec valeurs Nan , sans transformation (Hyperplot+300 -> 0.677)
- X_train_forest_imputed : Nan imputés avec Forest (Hyperplot+300 -> 0.673)
- X__Mice_imputed : Nan imputés avec Mice  (Hyperplot+300 -> 0.672)
- X_poly_2_No_nan : Nan imputés avec Forest + polynomes et interractions  (Hyperplot+300 -> 0.657)
- X_train_RandomUnderSampler , y_train_RandomUnderSampler : données parfaitement équilibrées mais moins importantes. (Hyperplot+300 -> 0.65) 
- X_train_del : imputation will bring a lot of noise (Hyperplot+300 -> 0.672, 1000 -> 0.673)
- X_train_poly2_outnan : polynomes sans OP100 (nan values) (Hyperplot+300 -> 0.657)

Les tests nous montrent que quelque soit la méthode d'imputation retenue , il y a perte de performances. Ceci est très probablement du  au fait que la création de bruit par les méthodes d'imputation.

In [11]:
import pandas as pd
import numpy as np
data_x = pd.read_csv('traininginputs.csv')
data_y = pd.read_csv('trainingoutput.csv')
data = pd.merge(data_x, data_y, on='PROC_TRACEINFO', how ='inner')
data = data.drop('PROC_TRACEINFO', axis =1 )
data['Binar OP130_Resultat_Global_v'] = data['Binar OP130_Resultat_Global_v'].astype('category')
X_train = data.drop('Binar OP130_Resultat_Global_v', axis =1)
y_train = data['Binar OP130_Resultat_Global_v']
X_train = X_train.drop('OP100_Capuchon_insertion_mesure', axis =1)

In [12]:
X_train.columns

Index(['OP070_V_1_angle_value', 'OP090_SnapRingPeakForce_value',
       'OP070_V_2_angle_value', 'OP120_Rodage_I_mesure_value',
       'OP090_SnapRingFinalStroke_value', 'OP110_Vissage_M8_torque_value',
       'OP120_Rodage_U_mesure_value', 'OP070_V_1_torque_value',
       'OP090_StartLinePeakForce_value', 'OP110_Vissage_M8_angle_value',
       'OP090_SnapRingMidPointForce_val', 'OP070_V_2_torque_value'],
      dtype='object')

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


In [5]:
import optuna
import warnings
warnings.filterwarnings('ignore')
from optuna import Trial, create_study, study

In [13]:
# Créer un objet Optuna Study
study = create_study(direction="maximize")

class_weights = {0: 1, 1: 1}

# Définition de l'espace de recherche
def objective(trial: Trial):
    n_estimators = trial.suggest_int("n_estimators", 10, 400)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    min_weight_fraction_leaf = trial.suggest_uniform('min_weight_fraction_leaf', 0.0, 0.5)


    # Créer un modèle RandomForestClassifier avec les paramètres suggérés
    clf = RandomForestClassifier(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  min_samples_split=min_samples_split,
                                  min_samples_leaf=min_samples_leaf,
                                  min_weight_fraction_leaf= min_weight_fraction_leaf,
                                  class_weight = class_weights, 
                                  n_jobs = -1)
    
    # Évaluation à l'aide de la validation croisée avec ROC AUC comme métrique
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')
    return scores.mean()

# Création de l'étude Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3000)

# Affichage des meilleurs hyperparamètres et de la meilleure ROC AUC moyenne
best_params = study.best_params
best_auc = study.best_value

[I 2023-11-04 15:19:02,327] A new study created in memory with name: no-name-2aa82ebb-ab1d-4250-9467-674ba63bf5fa
[I 2023-11-04 15:19:02,329] A new study created in memory with name: no-name-d52b9f80-d1a1-48cb-ad77-ce936eb2721a
[I 2023-11-04 15:19:04,536] Trial 0 finished with value: 0.6381364379124117 and parameters: {'n_estimators': 67, 'max_depth': 4, 'min_samples_split': 9, 'min_samples_leaf': 7, 'min_weight_fraction_leaf': 0.23576444120921092}. Best is trial 0 with value: 0.6381364379124117.
[I 2023-11-04 15:19:06,111] Trial 1 finished with value: 0.6147061783296035 and parameters: {'n_estimators': 153, 'max_depth': 6, 'min_samples_split': 6, 'min_samples_leaf': 5, 'min_weight_fraction_leaf': 0.3869950248609066}. Best is trial 0 with value: 0.6381364379124117.
[I 2023-11-04 15:19:07,272] Trial 2 finished with value: 0.6475781216306228 and parameters: {'n_estimators': 37, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.003150992532411967

In [ ]:
best_auc

0.669919158907615

In [ ]:
optuna.visualization.plot_optimization_history(study)